<a href="https://colab.research.google.com/github/jtviegas-sandbox/datascience/blob/main/notebooks/findata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install fredapi pandas_datareader

In [4]:
from fredapi import Fred
import pandas_datareader as pdr

In [5]:
from google.colab import userdata
FRED_KEY=userdata.get('FRED_API_KEY')

In [6]:
fred = Fred(api_key=FRED_KEY)

## cpi

In [8]:
cpi_us = fred.get_series("CPALTT01USM657N")
cpi_us

1955-02-01         NaN
1955-03-01         NaN
1955-04-01         NaN
1955-05-01         NaN
1955-06-01         NaN
                ...   
2023-06-01    0.322891
2023-07-01    0.190752
2023-08-01    0.436716
2023-09-01    0.248513
2023-10-01   -0.038338
Length: 825, dtype: float64